In [3]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense,Conv1D,LSTM,GRU,Flatten,MaxPooling2D,Conv2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition  import PCA
from sklearn.preprocessing import LabelEncoder

In [4]:
data = pd.read_csv("/kaggle/input/sky-cast-margazhi-25/train.csv")
data.head(5)

,Serial Number,Unnamed: 0.1,Unnamed: 0,Company Name,Location,Datum,Detail,Status Rocket,Rocket,Status Mission
0,3677,3676,3676,RVSN USSR,"Site 41/1, Plesetsk Cosmodrome, Russia","Thu Apr 18, 1968 10:30 UTC",Voskhod | Cosmos 214,StatusRetired,NaN,1
1,64,63,63,CASC,"LC-101, Wenchang Satellite Launch Center, China","Fri Dec 27, 2019 12:45 UTC",Long March 5 | Shijian-20,StatusActive,NaN,1
2,3930,3929,3929,RVSN USSR,"Site 86/1, Kapustin Yar, Russia","Thu Nov 04, 1965 05:31 UTC",Cosmos-2I (63SM) | Cosmos 95,StatusRetired,NaN,1
3,2035,2034,2034,RVSN USSR,"Site 41/1, Plesetsk Cosmodrome, Russia","Sat Nov 15, 1986 21:35 UTC",Molniya-M /Block ML | Molniya-1 n†­127,StatusRetired,NaN,1
4,1310,1309,1309,VKS RF,"Site 32/1, Plesetsk Cosmodrome, Russia","Mon Jun 15, 1998 22:58 UTC",Tsyklon-3 | Cosmos 2352 to 2357,StatusRetired,NaN,0


In [5]:
data.columns

Index(['Serial Number', 'Unnamed: 0.1', 'Unnamed: 0', 'Company Name',
       'Location', 'Datum', 'Detail', 'Status Rocket', ' Rocket',
       'Status Mission'],
      dtype='object')

In [6]:
data.info

<bound method DataFrame.info of       Serial Number  Unnamed: 0.1  Unnamed: 0      Company Name  \
0              3677          3676        3676         RVSN USSR   
1                64            63          63              CASC   
2              3930          3929        3929         RVSN USSR   
3              2035          2034        2034         RVSN USSR   
4              1310          1309        1309            VKS RF   
...             ...           ...         ...               ...   
3454           3445          3444        3444         RVSN USSR   
3455            467           466         466       Arianespace   
3456           3093          3092        3092  General Dynamics   
3457           3773          3772        3772         RVSN USSR   
3458            861           860         860              CASC   

                                               Location  \
0                Site 41/1, Plesetsk Cosmodrome, Russia   
1       LC-101, Wenchang Satellite Launch Cen

In [7]:
data=data.drop("Unnamed: 0.1",axis=1)
data=data.drop("Serial Number",axis=1)
data=data.drop("Unnamed: 0",axis=1)
data.head(5)

,Company Name,Location,Datum,Detail,Status Rocket,Rocket,Status Mission
0,RVSN USSR,"Site 41/1, Plesetsk Cosmodrome, Russia","Thu Apr 18, 1968 10:30 UTC",Voskhod | Cosmos 214,StatusRetired,NaN,1
1,CASC,"LC-101, Wenchang Satellite Launch Center, China","Fri Dec 27, 2019 12:45 UTC",Long March 5 | Shijian-20,StatusActive,NaN,1
2,RVSN USSR,"Site 86/1, Kapustin Yar, Russia","Thu Nov 04, 1965 05:31 UTC",Cosmos-2I (63SM) | Cosmos 95,StatusRetired,NaN,1
3,RVSN USSR,"Site 41/1, Plesetsk Cosmodrome, Russia","Sat Nov 15, 1986 21:35 UTC",Molniya-M /Block ML | Molniya-1 n†­127,StatusRetired,NaN,1
4,VKS RF,"Site 32/1, Plesetsk Cosmodrome, Russia","Mon Jun 15, 1998 22:58 UTC",Tsyklon-3 | Cosmos 2352 to 2357,StatusRetired,NaN,0


In [8]:
data["Company Name"].value_counts()

Company Name
RVSN USSR           1433
Arianespace          223
General Dynamics     211
CASC                 186
NASA                 166
VKS RF               152
US Air Force         128
ULA                  114
Boeing               111
Martin Marietta       91
SpaceX                71
MHI                   69
Lockheed              66
Northrop              63
ISRO                  61
Roscosmos             46
ILS                   35
Sea Launch            30
ISAS                  24
Kosmotras             15
US Navy               14
Eurockot              12
ISA                   12
ESA                   12
Blue Origin           11
Rocket Lab             9
ExPace                 8
CNES                   7
IAI                    7
Land Launch            7
JAXA                   6
ASI                    6
AMBA                   6
MITT                   6
Arm??e de l'Air        4
CASIC                  4
UT                     4
KCST                   4
Exos                   3
CECLES      

In [9]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()

data["Company Name"]=encoder.fit_transform(data["Company Name"])

data["Status Rocket"]=encoder.fit_transform(data["Status Rocket"])

data.head(5)

,Company Name,Location,Datum,Detail,Status Rocket,Rocket,Status Mission
0,37,"Site 41/1, Plesetsk Cosmodrome, Russia","Thu Apr 18, 1968 10:30 UTC",Voskhod | Cosmos 214,1,NaN,1
1,7,"LC-101, Wenchang Satellite Launch Center, China","Fri Dec 27, 2019 12:45 UTC",Long March 5 | Shijian-20,0,NaN,1
2,37,"Site 86/1, Kapustin Yar, Russia","Thu Nov 04, 1965 05:31 UTC",Cosmos-2I (63SM) | Cosmos 95,1,NaN,1
3,37,"Site 41/1, Plesetsk Cosmodrome, Russia","Sat Nov 15, 1986 21:35 UTC",Molniya-M /Block ML | Molniya-1 n†­127,1,NaN,1
4,49,"Site 32/1, Plesetsk Cosmodrome, Russia","Mon Jun 15, 1998 22:58 UTC",Tsyklon-3 | Cosmos 2352 to 2357,1,NaN,0


In [10]:
data["Location"].value_counts().unique

<bound method Series.unique of Location
Site 31/6, Baikonur Cosmodrome, Kazakhstan          192
Site 132/1, Plesetsk Cosmodrome, Russia             173
Site 43/4, Plesetsk Cosmodrome, Russia              173
Site 41/1, Plesetsk Cosmodrome, Russia              163
Site 1/5, Baikonur Cosmodrome, Kazakhstan           152
                                                   ... 
Site 110/37, Baikonur Cosmodrome, Kazakhstan          1
LC-576A1, Vandenberg AFB, California, USA             1
Site 110/38, Baikonur Cosmodrome, Kazakhstan          1
LC-11, Cape Canaveral AFS, Florida, USA               1
NB-52B Carrier, Cape Canaveral AFS, Florida, USA      1
Name: count, Length: 134, dtype: int64>

In [11]:
data["Status Rocket"].value_counts()

Status Rocket
1    2844
0     615
Name: count, dtype: int64

In [12]:
data[' Rocket'] = data[' Rocket'].str.replace(',', '').str.strip().astype(float)


In [13]:
data[" Rocket"][0:20]

0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
5        NaN
6        NaN
7        NaN
8     450.00
9        NaN
10       NaN
11       NaN
12       NaN
13    450.00
14       NaN
15       NaN
16     64.68
17       NaN
18       NaN
19       NaN
Name:  Rocket, dtype: float64

In [14]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(missing_values=np.nan, strategy='median')

data[" Rocket"] = imputer.fit_transform(data[[" Rocket"]])

In [15]:
pd.options.mode.copy_on_write = True
k = 0
for i in data["Detail"]:
    j = i.split("|")
    data.loc[k, "rocket_name"] = j[0]
    data.loc[k, "sat_name"] = j[1]
    k += 1

data=data.drop("Detail",axis=1)
data.head(5)

,Company Name,Location,Datum,Status Rocket,Rocket,Status Mission,rocket_name,sat_name
0,37,"Site 41/1, Plesetsk Cosmodrome, Russia","Thu Apr 18, 1968 10:30 UTC",1,63.23,1,Voskhod,Cosmos 214
1,7,"LC-101, Wenchang Satellite Launch Center, China","Fri Dec 27, 2019 12:45 UTC",0,63.23,1,Long March 5,Shijian-20
2,37,"Site 86/1, Kapustin Yar, Russia","Thu Nov 04, 1965 05:31 UTC",1,63.23,1,Cosmos-2I (63SM),Cosmos 95
3,37,"Site 41/1, Plesetsk Cosmodrome, Russia","Sat Nov 15, 1986 21:35 UTC",1,63.23,1,Molniya-M /Block ML,Molniya-1 n†­127
4,49,"Site 32/1, Plesetsk Cosmodrome, Russia","Mon Jun 15, 1998 22:58 UTC",1,63.23,0,Tsyklon-3,Cosmos 2352 to 2357


In [16]:

def standardize_datum(value):
    if len(value.split()) == 4:  
        return f"{value} 00:00 UTC"
    return value

# Standardize the 'Datum' āā
data['Datum'] = data['Datum'].apply(standardize_datum)


try:
    data['Datum'] = pd.to_datetime(data['Datum'], format="%a %b %d, %Y %H:%M %Z")
    conversion_status = "Conversion successful."
except Exception as e:
    conversion_status = f"Conversion failed: {e}"

conversion_status


'Conversion successful.'

In [17]:
data["year"]=data["Datum"].dt.year
data["month"]=data["Datum"].dt.month
data["day"]=data["Datum"].dt.day
data["hour"]=data["Datum"].dt.hour

In [18]:
data=data.drop("Datum",axis=1)
data.head(5)

,Company Name,Location,Status Rocket,Rocket,Status Mission,rocket_name,sat_name,year,month,day,hour
0,37,"Site 41/1, Plesetsk Cosmodrome, Russia",1,63.23,1,Voskhod,Cosmos 214,1968,4,18,10
1,7,"LC-101, Wenchang Satellite Launch Center, China",0,63.23,1,Long March 5,Shijian-20,2019,12,27,12
2,37,"Site 86/1, Kapustin Yar, Russia",1,63.23,1,Cosmos-2I (63SM),Cosmos 95,1965,11,4,5
3,37,"Site 41/1, Plesetsk Cosmodrome, Russia",1,63.23,1,Molniya-M /Block ML,Molniya-1 n†­127,1986,11,15,21
4,49,"Site 32/1, Plesetsk Cosmodrome, Russia",1,63.23,0,Tsyklon-3,Cosmos 2352 to 2357,1998,6,15,22


In [19]:
location_split = data['Location'].str.split(',', expand=True)
data['center'] = location_split.iloc[:, 1].str.strip()
data['country'] = location_split.iloc[:, -1].str.strip()
data['site'] = location_split.iloc[:, 0].str.strip()
data['center'].value_counts()

center
Plesetsk Cosmodrome                                  1027
Baikonur Cosmodrome                                   550
Cape Canaveral AFS                                    528
Vandenberg AFB                                        356
Guiana Space Centre                                   241
Kennedy Space Center                                  150
Kapustin Yar                                           91
Jiuquan Satellite Launch Center                        84
Tanegashima Space Center                               70
Xichang Satellite Launch Center                        67
Satish Dhawan Space Centre                             61
Taiyuan Satellite Launch Center                        35
Kiritimati Launch Area                                 30
Uchinoura Space Center                                 27
Wallops Flight Facility                                26
Semnan Space Center                                    12
West Texas                                             11
M?hia 

In [20]:
data=data.drop("Location",axis=1)

In [21]:
data.head(5)

,Company Name,Status Rocket,Rocket,Status Mission,rocket_name,sat_name,year,month,day,hour,center,country,site
0,37,1,63.23,1,Voskhod,Cosmos 214,1968,4,18,10,Plesetsk Cosmodrome,None,Site 41/1
1,7,0,63.23,1,Long March 5,Shijian-20,2019,12,27,12,Wenchang Satellite Launch Center,None,LC-101
2,37,1,63.23,1,Cosmos-2I (63SM),Cosmos 95,1965,11,4,5,Kapustin Yar,None,Site 86/1
3,37,1,63.23,1,Molniya-M /Block ML,Molniya-1 n†­127,1986,11,15,21,Plesetsk Cosmodrome,None,Site 41/1
4,49,1,63.23,0,Tsyklon-3,Cosmos 2352 to 2357,1998,6,15,22,Plesetsk Cosmodrome,None,Site 32/1


In [22]:
data["center"]=encoder.fit_transform(data["center"])

data["rocket_name"]=encoder.fit_transform(data["rocket_name"])

data["sat_name"]=encoder.fit_transform(data["sat_name"])

data["country"]=encoder.fit_transform(data["country"])

data["site"]=encoder.fit_transform(data["site"])

In [23]:
y_data=data["Status Mission"]
x_data=data.drop("Status Mission",axis=1)

In [24]:
x_data.head(10)

,Company Name,Status Rocket,Rocket,rocket_name,sat_name,year,month,day,hour,center,country,site
0,37,1,63.23,322,702,1968,4,18,10,21,3,106
1,7,0,63.23,171,3011,2019,12,27,12,38,3,21
2,37,1,63.23,74,1345,1965,11,4,5,11,3,115
3,37,1,63.23,183,2297,1986,11,15,21,21,3,106
4,49,1,63.23,315,776,1998,6,15,22,21,3,102
5,37,1,63.23,267,1217,1975,9,29,4,1,3,82
6,37,1,63.23,322,851,1969,3,6,12,21,3,106
7,37,1,63.23,324,915,1964,7,15,11,1,3,101
8,33,1,450.00,272,2845,2002,11,24,0,13,2,35
9,3,1,63.23,14,3043,1993,11,20,1,7,1,5


In [25]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test=train_test_split(x_data,y_data,test_size=0.2,random_state=42)

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(2767, 12)
(2767,)
(692, 12)
(692,)


In [26]:
data["Status Mission"]

0       1
1       1
2       1
3       1
4       0
       ..
3454    1
3455    1
3456    1
3457    1
3458    1
Name: Status Mission, Length: 3459, dtype: int64

In [27]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_train = scaler.fit_transform(x_train)
X_test = scaler.transform(x_test)

x_train=np.array(x_train)
x_test=np.array(x_test)

x_train=x_train.reshape(-1,12,1)
x_test=x_test.reshape(-1,12,1)


In [28]:
'''model = Sequential([
    Dense(128,activation="relu",input_shape=(12,)),
    Dense(64,activation="relu"),
    Dense(32,activation="relu"),
    Dense(16,activation="relu"),
    Dense(1,activation="sigmoid")
])


model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])'''

'model = Sequential([\n    Dense(128,activation="relu",input_shape=(12,)),\n    Dense(64,activation="relu"),\n    Dense(32,activation="relu"),\n    Dense(16,activation="relu"),\n    Dense(1,activation="sigmoid")\n])\n\n\nmodel.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])'

In [29]:
x_train.shape

(2767, 12, 1)

In [49]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.regularizers import l2

from tensorflow.keras.layers import BatchNormalization, Bidirectional

model = Sequential([
    Bidirectional(LSTM(128, activation="tanh", return_sequences=True, kernel_regularizer=l2(0.01)), input_shape=(12, 1)),
    Dropout(0.4),
    BatchNormalization(),

    Bidirectional(LSTM(128, activation="tanh", return_sequences=True, kernel_regularizer=l2(0.01))),
    Dropout(0.2),
    BatchNormalization(),

    LSTM(64, activation="tanh", kernel_regularizer=l2(0.01)),
    Dropout(0.4),
    BatchNormalization(),

    Dense(64, activation="relu"),
    Dropout(0.1),
    Dense(32, activation="relu"),
    Dropout(0.1),
    Dense(1, activation="sigmoid")
])

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Implement early stopping and learning rate reduction
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

#early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)

# Train model
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
87/87 ━━━━━━━━━━━━━━━━━━━━ 16s 71ms/step - accuracy: 0.7989 - loss: 7.8396 - val_accuracy: 0.9104 - val_loss: 3.3510
Epoch 2/10
87/87 ━━━━━━━━━━━━━━━━━━━━ 5s 59ms/step - accuracy: 0.8987 - loss: 2.6995 - val_accuracy: 0.9104 - val_loss: 1.5398
Epoch 3/10
87/87 ━━━━━━━━━━━━━━━━━━━━ 5s 59ms/step - accuracy: 0.8915 - loss: 1.3222 - val_accuracy: 0.9104 - val_loss: 0.9135
Epoch 4/10
87/87 ━━━━━━━━━━━━━━━━━━━━ 5s 59ms/step - accuracy: 0.8951 - loss: 0.8343 - val_accuracy: 0.9104 - val_loss: 0.6894
Epoch 5/10
87/87 ━━━━━━━━━━━━━━━━━━━━ 10s 59ms/step - accuracy: 0.8955 - loss: 0.6151 - val_accuracy: 0.9104 - val_loss: 0.5158
Epoch 6/10
87/87 ━━━━━━━━━━━━━━━━━━━━ 6s 65ms/step - accuracy: 0.8989 - loss: 0.4961 - val_accuracy: 0.9104 - val_loss: 0.4188
Epoch 7/10
87/87 ━━━━━━━━━━━━━━━━━━━━ 10s 60ms/step - accuracy: 0.8981 - loss: 0.4417 - val_accuracy: 0.9104 - val_loss: 0.4170
Epoch 8/10
87/87 ━━━━━━━━━━━━━━━━━━━━ 5s 57ms/step - accuracy: 0.8972 - loss: 0.4273 - val_accuracy: 0.9104 

In [50]:
prediction_test=model.predict(x_test)


from sklearn.metrics import f1_score

threshold = 0.5
y_pred = [1 if prob >= threshold else 0 for prob in prediction_test ]


f1 = f1_score(y_pred,y_test)  # Use 'binary' for binary classification
print(f"F1 Score: {f1}")

22/22 ━━━━━━━━━━━━━━━━━━━━ 2s 57ms/step
F1 Score: 0.9531013615733737


In [32]:
testing=pd.read_csv("/kaggle/input/sky-cast-margazhi-25/test.csv")
testing.head(10)
testing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 865 entries, 0 to 864
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Serial Number  865 non-null    int64 
 1   Unnamed: 0.1   865 non-null    int64 
 2   Unnamed: 0     865 non-null    int64 
 3   Company Name   865 non-null    object
 4   Location       865 non-null    object
 5   Datum          865 non-null    object
 6   Detail         865 non-null    object
 7   Status Rocket  865 non-null    object
 8    Rocket        213 non-null    object
dtypes: int64(3), object(6)
memory usage: 60.9+ KB


In [33]:
testing=testing.drop("Serial Number",axis=1)
testing=testing.drop("Unnamed: 0.1",axis=1)
testing=testing.drop("Unnamed: 0",axis=1)

In [34]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()

testing["Company Name"]=encoder.fit_transform(testing["Company Name"])

testing["Status Rocket"]=encoder.fit_transform(testing["Status Rocket"])

In [35]:
testing[' Rocket'] = testing[' Rocket'].str.replace(',', '').str.strip().astype(float)

In [36]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(missing_values=np.nan, strategy='mean')

testing[" Rocket"] = imputer.fit_transform(testing[[" Rocket"]])

In [37]:
pd.options.mode.copy_on_write = True
k = 0
for i in testing["Detail"]:
    j = i.split("|")
    testing.loc[k, "rocket_name"] = j[0]
    testing.loc[k, "sat_name"] = j[1]
    k += 1

testing=testing.drop("Detail",axis=1)
testing.head(5)

,Company Name,Location,Datum,Status Rocket,Rocket,rocket_name,sat_name
0,30,"LC-39A, Kennedy Space Center, Florida, USA","Thu Jul 25, 1985 21:00 UTC",1,450.000000,Space Shuttle Challenger,STS-51-F
1,37,"SLC-40, Cape Canaveral AFS, Florida, USA","Tue Jun 30, 2020 20:10 UTC",0,50.000000,Falcon 9 Block 5,GPS III SV03
2,33,"Site 133/3, Plesetsk Cosmodrome, Russia","Tue Nov 21, 1967 14:29 UTC",1,124.665211,Cosmos-2I (63SM),Cosmos 191
3,38,"SLC-41, Cape Canaveral AFS, Florida, USA","Mon Nov 18, 2013 18:28 UTC",0,109.000000,Atlas V 401,MAVEN
4,33,"Site 1/5, Baikonur Cosmodrome, Kazakhstan","Mon Oct 10, 1960 14:27 UTC",1,124.665211,Molniya,1M n†­1 (Marsnik 1)


In [38]:
def standardize_datum(value):
    if len(value.split()) == 4:  
        return f"{value} 00:00 UTC"
    return value

# Standardize the 'Datum' column
testing['Datum'] = testing['Datum'].apply(standardize_datum)


try:
    testing['Datum'] = pd.to_datetime(testing['Datum'], format="%a %b %d, %Y %H:%M %Z")
    conversion_status = "Conversion successful."
except Exception as e:
    conversion_status = f"Conversion failed: {e}"

conversion_status

'Conversion successful.'

In [39]:
testing["year"]=testing["Datum"].dt.year
testing["month"]=testing["Datum"].dt.month
testing["day"]=testing["Datum"].dt.day
testing["hour"]=testing["Datum"].dt.hour
testing=testing.drop("Datum",axis=1)
testing.head(5)

,Company Name,Location,Status Rocket,Rocket,rocket_name,sat_name,year,month,day,hour
0,30,"LC-39A, Kennedy Space Center, Florida, USA",1,450.000000,Space Shuttle Challenger,STS-51-F,1985,7,25,21
1,37,"SLC-40, Cape Canaveral AFS, Florida, USA",0,50.000000,Falcon 9 Block 5,GPS III SV03,2020,6,30,20
2,33,"Site 133/3, Plesetsk Cosmodrome, Russia",1,124.665211,Cosmos-2I (63SM),Cosmos 191,1967,11,21,14
3,38,"SLC-41, Cape Canaveral AFS, Florida, USA",0,109.000000,Atlas V 401,MAVEN,2013,11,18,18
4,33,"Site 1/5, Baikonur Cosmodrome, Kazakhstan",1,124.665211,Molniya,1M n†­1 (Marsnik 1),1960,10,10,14


In [40]:
location_split = testing['Location'].str.split(',', expand=True)
testing['center'] = location_split.iloc[:, 1].str.strip()
testing=testing.drop("Location",axis=1)

In [41]:
testing["center"]=encoder.fit_transform(testing["center"])

testing["rocket_name"]=encoder.fit_transform(testing["rocket_name"])

testing["sat_name"]=encoder.fit_transform(testing["sat_name"])

In [42]:
testing.head(10)
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
testing = scaler.fit_transform(testing)
testing=np.array(testing)


testing=testing.reshape(-1,10,1)


In [43]:
prediction=model.predict(testing)

28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step


In [44]:
len(prediction)

865

In [45]:
thresholded_values = (prediction >= 0.5).astype(int)

hello=pd.read_csv("/kaggle/input/sky-cast-margazhi-25/test.csv")
# Create a dataframe for the submission
submission_df = pd.DataFrame({
    'Serial Number': hello["Serial Number"] ,
    'Status Mission': thresholded_values.flatten()
})

submission_df.to_csv('submission.csv', index=False)


In [46]:
submission=pd.read_csv("/kaggle/working/submission.csv")
submission.tail(10)

,Serial Number,Status Mission
855,991,1
856,2851,1
857,2269,1
858,1878,1
859,140,1
860,1004,1
861,2393,1
862,3757,1
863,267,1
864,2481,1


In [47]:
submission.head(5)

,Serial Number,Status Mission
0,2122,1
1,18,1
2,3716,1
3,585,1
4,4249,1


In [48]:
submission.to_csv('submission.csv', index=False)